<a href="https://colab.research.google.com/github/tlsgptj/ingognunggi/blob/main/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

수정된 코드

In [ ]:
import queue

# 상태를 나타내는 클래스, f(n) 값을 저장한다.
class State:
    def __init__(self, board, goal, depth=0):
        self.board = board         # 현재의 보드 상태
        self.depth = depth         # 깊이
        self.goal = goal           # 목표 상태

    # i1과 i2를 교환하여서 새로운 상태를 반환한다.
    def get_new_board(self, i1, i2, depth):
        new_board = self.board[:]
        new_board[i1], new_board[i2] = new_board[i2], new_board[i1]
        return State(new_board, self.goal, depth)

    # 자식 노드를 확장하여서 리스트에 저장하여서 반환한다.
    def expand(self, moves):
        result = []
        i = self.board.index(0)      # 숫자 0(빈칸)의 위치를 찾는다.
        if not i in [0, 3, 6]:      # LEFT 연산자
            result.append(self.get_new_board(i, i-1, moves))
        if not i in [0, 1, 2]:      # UP 연산자
            result.append(self.get_new_board(i, i-3, moves))
        if not i in [2, 5, 8]:      # RIGHT 연산자
            result.append(self.get_new_board(i, i+1, moves))
        if not i in [6, 7, 8]:      # DOWN 연산자
            result.append(self.get_new_board(i, i+3, moves))
        return result

    # f(n)을 계산하여 반환한다.
    def f(self):
        return self.h() + self.g()

    # 맨해튼 거리 휴리스틱 함수 h(n)을 계산하여 반환한다.
    def h(self):
        score = 0
        for i in range(9):
            if self.board[i] != 0:
                goal_index = self.goal.index(self.board[i])
                current_row, current_col = i // 3, i % 3
                goal_row, goal_col = goal_index // 3, goal_index % 3
                score += abs(current_row - goal_row) + abs(current_col - goal_col)
        return score

    # 시작 노드로부터의 깊이를 반환한다.
    def g(self):
        return self.depth

    def __eq__(self, other):
        return self.board == other.board

    def __ne__(self, other):
        return self.board != other.board

    # 상태와 상태를 비교하기 위하여 less than 연산자를 정의한다.
    def __lt__(self, other):
        return self.f() < other.f()

    def __gt__(self, other):
        return self.f() > other.f()

    # 객체를 출력할 때 사용한다.
    def __str__(self):
        return f"f(n)={self.f()} h(n)={self.h()} g(n)={self.g()}\n" + \
               str(self.board[:3]) + "\n" + \
               str(self.board[3:6]) + "\n" + \
               str(self.board[6:]) + "\n"

# 초기 상태
puzzle = [2, 8, 3,
          1, 6, 4,
          7, 0, 5]
# 목표 상태
goal = [1, 2, 3,
        8, 0, 4,
        7, 6, 5]

# open 리스트는 우선순위 큐로 생성한다.
open_queue = queue.PriorityQueue()
open_queue.put(State(puzzle, goal))

closed_queue = []
depth = 0
count = 0

while not open_queue.empty():
    current = open_queue.get()
    count += 1
    print(count)
    print(current)
    if current.board == goal:
        print("탐색 성공")
        break
    depth = current.depth + 1
    for state in current.expand(depth):
        if state not in closed_queue and state not in open_queue.queue:
            open_queue.put(state)
    closed_queue.append(current)
else:
    print('탐색 실패')


3번 코드

In [ ]:
import queue
import random

# 상태를 나타내는 클래스, f(n) 값을 저장한다.
class State:
    def __init__(self, board, goal, depth=0, heuristic='h1'):
        self.board = board         # 현재의 보드 상태
        self.depth = depth         # 깊이
        self.goal = goal           # 목표 상태
        self.heuristic = heuristic # 사용할 휴리스틱 선택 ('h1' 또는 'h2')

    # i1과 i2를 교환하여서 새로운 상태를 반환한다.
    def get_new_board(self, i1, i2, depth):
        new_board = self.board[:]
        new_board[i1], new_board[i2] = new_board[i2], new_board[i1]
        return State(new_board, self.goal, depth, self.heuristic)

    # 자식 노드를 확장하여서 리스트에 저장하여서 반환한다.
    def expand(self, moves):
        result = []
        i = self.board.index(0)      # 숫자 0(빈칸)의 위치를 찾는다.
        if not i in [0, 3, 6]:      # LEFT 연산자
            result.append(self.get_new_board(i, i-1, moves))
        if not i in [0, 1, 2]:      # UP 연산자
            result.append(self.get_new_board(i, i-3, moves))
        if not i in [2, 5, 8]:      # RIGHT 연산자
            result.append(self.get_new_board(i, i+1, moves))
        if not i in [6, 7, 8]:      # DOWN 연산자
            result.append(self.get_new_board(i, i+3, moves))
        return result

    # f(n)을 계산하여 반환한다.
    def f(self):
        return self.h() + self.g()

    # 휴리스틱 함수 h(n)을 계산하여 반환한다.
    def h(self):
        if self.heuristic == 'h1':  # 잘못 놓인 타일의 개수
            return self.h1()
        elif self.heuristic == 'h2':  # 맨해튼 거리
            return self.h2()

    # 잘못 놓인 타일의 개수 계산 (h1)
    def h1(self):
        score = 0
        for i in range(9):
            if self.board[i] != 0 and self.board[i] != self.goal[i]:
                score += 1
        return score

    # 맨해튼 거리 계산 (h2)
    def h2(self):
        score = 0
        for i in range(9):
            if self.board[i] != 0:  # 빈칸(0)은 제외하고 계산
                goal_index = self.goal.index(self.board[i])
                current_row, current_col = i // 3, i % 3
                goal_row, goal_col = goal_index // 3, goal_index % 3
                score += abs(current_row - goal_row) + abs(current_col - goal_col)
        return score

    # 시작 노드로부터의 깊이를 반환한다.
    def g(self):
        return self.depth

    def __eq__(self, other):
        return self.board == other.board

    def __lt__(self, other):
        return self.f() < other.f()

    def __hash__(self):
        return hash(tuple(self.board))  # hash를 정의해 set에서 사용 가능하게 함

    def __str__(self):
        return f"f(n)={self.f()} h(n)={self.h()} g(n)={self.g()}\n" + \
               str(self.board[:3]) + "\n" + \
               str(self.board[3:6]) + "\n" + \
               str(self.board[6:]) + "\n"

# A* 탐색 알고리즘을 실행하는 함수
def a_star(puzzle, goal, heuristic, max_depth=50):
    open_queue = queue.PriorityQueue()
    open_queue.put(State(puzzle, goal, heuristic=heuristic))
    open_set = {tuple(puzzle)}  # open_queue에 들어간 상태를 추적
    closed_set = set()
    total_nodes = 0  # 탐색 중 생성된 모든 노드의 수

    while not open_queue.empty():
        current = open_queue.get()
        open_set.remove(tuple(current.board))

        # 탐색 깊이 제한
        if current.depth > max_depth:
            return total_nodes  # 최대 깊이를 초과하면 종료

        if current.board == goal:
            return total_nodes  # 탐색에 사용된 모든 노드 수 반환

        total_nodes += 1
        depth = current.depth + 1
        for state in current.expand(depth):
            state_tuple = tuple(state.board)
            if state_tuple not in closed_set and state_tuple not in open_set:
                open_queue.put(state)
                open_set.add(state_tuple)
        closed_set.add(tuple(current.board))
    return total_nodes  # 탐색 실패 시에도 노드 수 반환

# 목표 상태로부터 무작위 이동을 가해 초기 상태를 생성
def generate_random_puzzle(goal, num_moves=20):
    puzzle = goal[:]
    for _ in range(num_moves):
        i = puzzle.index(0)
        possible_moves = []
        if i not in [0, 3, 6]: possible_moves.append(i - 1)  # LEFT
        if i not in [0, 1, 2]: possible_moves.append(i - 3)  # UP
        if i not in [2, 5, 8]: possible_moves.append(i + 1)  # RIGHT
        if i not in [6, 7, 8]: possible_moves.append(i + 3)  # DOWN
        if possible_moves:
            swap_idx = random.choice(possible_moves)
            puzzle[i], puzzle[swap_idx] = puzzle[swap_idx], puzzle[i]
    return puzzle

# 목표 상태
goal = [1, 2, 3, 8, 0, 4, 7, 6, 5]

# 10개의 랜덤한 초기 상태 생성 (목표 상태에서 무작위로 이동)
random_puzzles = [generate_random_puzzle(goal) for _ in range(10)]

# 각 휴리스틱에 대해 탐색 비용 비교
h1_costs = []
h2_costs = []

for puzzle in random_puzzles:
    h1_costs.append(a_star(puzzle, goal, 'h1'))
    h2_costs.append(a_star(puzzle, goal, 'h2'))

# 결과 출력
print("h1 (잘못 놓인 타일의 개수) 탐색 비용:", h1_costs)
print("h2 (맨해튼 거리) 탐색 비용:", h2_costs)
print("h1 평균 탐색 비용:", sum(h1_costs) / len(h1_costs))
print("h2 평균 탐색 비용:", sum(h2_costs) / len(h2_costs))